In [1]:
from pyspark.sql import SparkSession
import findspark
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

findspark.init()
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Otodom Regression") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/22 12:48:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/22 12:48:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data_path = "Otodom_Flat_Listings.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

df.show(7)

+--------------------+---------+--------------------+-------+---------------+------+-------------------+--------+--------------------+----------------------+--------------------+------------------+-------------------+
|               Title|    Price|            Location|Surface|Number_of_Rooms| Floor|Finishing_Condition| Heating|       Parking_Space|Balcony_Garden_Terrace|                Link|       Voivodeship|               City|
+--------------------+---------+--------------------+-------+---------------+------+-------------------+--------+--------------------+----------------------+--------------------+------------------+-------------------+
|2 pokoje 47m2 po ...| 415000.0|ul. Marysińska, S...|   47.0|             2 |  NULL|    do zamieszkania|miejskie|garaż/miejsce par...|                balkon|https://www.otodo...|           Łódzkie|               Łódź|
|Właściciel- Ludwi...|2499000.0|ul. Ludwiki, Czys...|  105.0|             4 |   2/8|     do wykończenia|miejskie|garaż/miejsce p

In [3]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Surface: string (nullable = true)
 |-- Number_of_Rooms: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Finishing_Condition: string (nullable = true)
 |-- Heating: string (nullable = true)
 |-- Parking_Space: string (nullable = true)
 |-- Balcony_Garden_Terrace: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Voivodeship: string (nullable = true)
 |-- City: string (nullable = true)



In [4]:
indexer_location = StringIndexer(inputCol="Location", outputCol="Location_indexed", handleInvalid='skip')
indexer_voivodeship = StringIndexer(inputCol="Voivodeship", outputCol="Voivodeship_indexed", handleInvalid='skip')
indexer_city = StringIndexer(inputCol="City", outputCol="City_indexed", handleInvalid='skip')
indexer_condition = StringIndexer(inputCol="Finishing_Condition", outputCol="Finishing_Condition_indexed", handleInvalid='skip')

In [5]:
df_indexed = indexer_location.fit(df).transform(df)
df_indexed = indexer_voivodeship.fit(df_indexed).transform(df_indexed)
df_indexed = indexer_city.fit(df_indexed).transform(df_indexed)
df_indexed = indexer_condition.fit(df_indexed).transform(df_indexed)

In [6]:
encoder_location = OneHotEncoder(inputCol="Location_indexed", outputCol="Location_onehot")
encoder_voivodeship = OneHotEncoder(inputCol="Voivodeship_indexed", outputCol="Voivodeship_onehot")
encoder_city = OneHotEncoder(inputCol="City_indexed", outputCol="City_onehot")
encoder_condition = OneHotEncoder(inputCol="Finishing_Condition_indexed", outputCol="Condition_onehot")

In [7]:
df_encoded = encoder_location.fit(df_indexed).transform(df_indexed)
df_encoded = encoder_voivodeship.fit(df_encoded).transform(df_encoded)
df_encoded = encoder_city.fit(df_encoded).transform(df_encoded)
df_encoded = encoder_condition.fit(df_encoded).transform(df_encoded)

In [8]:
from pyspark.sql.functions import when, col, split, log

df_selected = df_encoded.select("Price", "Surface", "Number_of_Rooms", "Floor", "Location_onehot", 
                               "Voivodeship_onehot", "City_onehot", "Condition_onehot")

df_processed = df_selected.withColumn(
    "Floor",
    when(col("Floor").isNotNull(), split(col("Floor"), "/")[0].cast("double")).otherwise(None)
)

df_processed = df_processed.withColumn("Surface", col("Surface").cast("double"))
df_processed = df_processed.withColumn("Number_of_Rooms", col("Number_of_Rooms").cast("double"))

df_processed = df_processed.dropna(subset=["Floor", "Surface", "Number_of_Rooms", "Location_onehot", "Price"])

In [9]:
df_processed = df_processed.withColumn("Log_Price", log(col("Price")))
df_processed.show(5)

+---------+-------+---------------+-----+-------------------+------------------+-----------------+----------------+------------------+
|    Price|Surface|Number_of_Rooms|Floor|    Location_onehot|Voivodeship_onehot|      City_onehot|Condition_onehot|         Log_Price|
+---------+-------+---------------+-----+-------------------+------------------+-----------------+----------------+------------------+
|2499000.0|  105.0|            4.0|  2.0| (1849,[390],[1.0])|    (15,[0],[1.0])|  (235,[0],[1.0])|   (2,[1],[1.0])| 14.73140120981709|
| 649000.0|   60.0|            3.0|  4.0| (1849,[387],[1.0])|   (15,[13],[1.0])| (235,[51],[1.0])|   (2,[0],[1.0])|13.383187995686226|
|2108000.0|   78.3|            3.0|  4.0| (1849,[655],[1.0])|    (15,[7],[1.0])| (235,[90],[1.0])|   (2,[1],[1.0])| 14.56125018864339|
| 450180.0|   49.2|            2.0|  2.0| (1849,[964],[1.0])|    (15,[7],[1.0])| (235,[19],[1.0])|   (2,[0],[1.0])| 13.01740278176783|
| 220000.0|   48.4|            3.0|  4.0|(1849,[1737],[

In [10]:
assembler = VectorAssembler(inputCols=["Surface", "Number_of_Rooms", "Floor", 
                                       "Location_onehot", "Voivodeship_onehot", 
                                       "City_onehot", "Condition_onehot"], outputCol="features")
df_processed = assembler.transform(df_processed)
df_processed.show(5)

+---------+-------+---------------+-----+-------------------+------------------+-----------------+----------------+------------------+--------------------+
|    Price|Surface|Number_of_Rooms|Floor|    Location_onehot|Voivodeship_onehot|      City_onehot|Condition_onehot|         Log_Price|            features|
+---------+-------+---------------+-----+-------------------+------------------+-----------------+----------------+------------------+--------------------+
|2499000.0|  105.0|            4.0|  2.0| (1849,[390],[1.0])|    (15,[0],[1.0])|  (235,[0],[1.0])|   (2,[1],[1.0])| 14.73140120981709|(2104,[0,1,2,393,...|
| 649000.0|   60.0|            3.0|  4.0| (1849,[387],[1.0])|   (15,[13],[1.0])| (235,[51],[1.0])|   (2,[0],[1.0])|13.383187995686226|(2104,[0,1,2,390,...|
|2108000.0|   78.3|            3.0|  4.0| (1849,[655],[1.0])|    (15,[7],[1.0])| (235,[90],[1.0])|   (2,[1],[1.0])| 14.56125018864339|(2104,[0,1,2,658,...|
| 450180.0|   49.2|            2.0|  2.0| (1849,[964],[1.0])|   

In [11]:
import pyspark.sql.functions as F

def run_experiment(features, df_processed):
    if 'features' in df_processed.columns:
        df_processed = df_processed.drop('features')

    assembler = VectorAssembler(inputCols=features, outputCol="features")
    df_experiment = assembler.transform(df_processed)

    train_data, test_data = df_experiment.randomSplit([0.8, 0.2], seed=1234)

    lr = LinearRegression(featuresCol="features", labelCol="Log_Price", regParam=0.1)

    lrModel = lr.fit(train_data)

    predictions = lrModel.transform(test_data)

    predictions = predictions.withColumn("predicted_price", F.exp(predictions["prediction"]))

    predictions = predictions.withColumn("actual_price", F.exp(predictions["Log_Price"]))

    evaluator_rmse = RegressionEvaluator(labelCol="Price", predictionCol="predicted_price", metricName="rmse")
    evaluator_r2 = RegressionEvaluator(labelCol="Price", predictionCol="predicted_price", metricName="r2")
    evaluator_mse = RegressionEvaluator(labelCol="Price", predictionCol="predicted_price", metricName="mse")
    
    rmse = evaluator_rmse.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)
    mse = evaluator_mse.evaluate(predictions)

    return {"features": features, "RMSE": rmse, "R2": r2, "MSE": mse, "predictions": predictions}

In [12]:
experiments = [
    ["Surface", "Number_of_Rooms", "Floor"],
    ["Surface", "Number_of_Rooms", "Floor", "Location_onehot"],
    ["Surface", "Number_of_Rooms", "Floor", "Location_onehot", "Voivodeship_onehot"],
    ["Surface", "Number_of_Rooms", "Floor", "Location_onehot", "Voivodeship_onehot", "City_onehot"],
    ["Surface", "Number_of_Rooms", "Floor", "Location_onehot", "Voivodeship_onehot", "City_onehot", "Condition_onehot"]
]

results = []
for exp in experiments:
    result = run_experiment(exp, df_processed)
    results.append(result)

24/12/22 12:48:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/22 12:48:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/12/22 12:48:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
for result in results:
    print(f"Zestaw cech: {result['features']}")
    print(f"RMSE: {result['RMSE']}, R2: {result['R2']}, MSE: {result['MSE']}")
    print("Przykładowe przewidywania (faktyczna vs przewidywana cena):")
    
    sample_predictions = result["predictions"].select("actual_price", "predicted_price").show(5)
    print("-" * 40)

Zestaw cech: ['Surface', 'Number_of_Rooms', 'Floor']
RMSE: 9095680.037142925, R2: 0.0019193969553040757, MSE: 82731395338080.33
Przykładowe przewidywania (faktyczna vs przewidywana cena):
+------------------+------------------+
|      actual_price|   predicted_price|
+------------------+------------------+
| 99000.00000000006| 466742.6982482966|
|117300.00000000009|448136.15557321196|
|178999.99999999994| 410955.0548019228|
|183000.00000000003|492830.50377878436|
|185000.00000000006| 365930.8274557564|
+------------------+------------------+
only showing top 5 rows

----------------------------------------
Zestaw cech: ['Surface', 'Number_of_Rooms', 'Floor', 'Location_onehot']
RMSE: 4918848.2176086195, R2: 0.708108055596752, MSE: 24195067787871.49
Przykładowe przewidywania (faktyczna vs przewidywana cena):
+------------------+------------------+
|      actual_price|   predicted_price|
+------------------+------------------+
| 99000.00000000006|487496.57687131857|
|117300.00000000009|13